In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# load training and test data
users_train = pd.read_csv('data/train/users.csv')
problems_train = pd.read_csv('data/train/problems.csv')
submissions_train = pd.read_csv('data/train/submissions.csv')

users_test = pd.read_csv('data/test/users.csv')
problems_test = pd.read_csv('data/test/problems.csv')
submissions_test = pd.read_csv('data/test/test.csv')

In [3]:
# create response variable
y = ((submissions_train.solved_status=='SO') | (submissions_train.solved_status=='UK')) * 1.

In [4]:
# prepare training data
submissions_users_train = pd.merge(submissions_train, users_train, on='user_id')
user_problem_train = pd.merge(submissions_users_train, problems_train, on='problem_id')

# prepare test data
submissions_users_test = pd.merge(submissions_test, users_test, on='user_id')
user_problem_test = pd.merge(submissions_users_test, problems_test, on='problem_id')

## Sample data

In [5]:
from sklearn.cross_validation import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(y, train_size=0.1)
train_index, test_index = next(iter(sss))

user_problem_train_sample = user_problem_train.iloc[train_index]
y_sample = y.iloc[train_index]

## Prepare a very simple model based on just the accuracy of the problem

In [6]:
%run scripts/features.py
%run scripts/models.py
%run scripts/eval.py

In [7]:
model = build_random_forest_classifier(user_problem_train, user_problem_test)

In [8]:
import time

In [ ]:
# fit model
start_time = time.time()
model.fit(user_problem_train, y)
elapsed_time = time.time() - start_time

In [ ]:
print 'It took %f to train the model ' %(elapsed_time)

In [ ]:
predictions  = model.predict(user_problem_test)

In [ ]:
# submissions
submission_df = pd.DataFrame({'Id': range(1, len(user_problem_test) + 1), 'solved_status': predictions.astype(int)})
submission_df.to_csv('submissions/second_rf.csv', index=False)

In [ ]:
user_problem_train.level.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()
lbl.fit(user_problem_train.level)

levels = lbl.transform(user_problem_train.level)

In [ ]:
pd.Series(levels).value_counts()